In [1]:
import streamlit as st
import os
import requests
from utils.save_recipes import save_recipe_as_pdf, get_recipe_pdf_download_link
from dotenv import load_dotenv
load_dotenv()
import openai
from google.oauth2 import service_account
from spellchecker import SpellChecker
# Set your API key
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])

In [2]:
from google.cloud import vision

def detect_document(uploaded_image):

    client = vision.ImageAnnotatorClient(credentials=credentials)

    #with io.BytesIO(uploaded_image) as image_file:
    #    content = image_file

    #image = vision.Image(uploaded_image)

    response = client.document_text_detection(image=uploaded_image)

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))
    
    response_text = response.full_text_annotation.text


    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    
    return response_text